In [2]:
# from newspaper import Article
# import nltk

# url = 'https://www.yahoo.com/news/labor-board-accuses-apple-suppressing-222846876.html'
# article = Article(url)

# try:
#     article.download()
#     article.parse()
#     article.nlp()  # Run NLP after parsing

#     # Print results
#     print("Summary:", article.summary)
# except Exception as e:
#     print("An error occurred:", e)

In [3]:
from newspaper import Article
import newspaper
import pandas as pd
# from langdetect import detect
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

In [5]:
def scrape_tesla_news(url):
    """
    Scrapes Tesla news from the given Yahoo Finance URL.

    Args:
        url: The URL of the Yahoo Finance page for Tesla news.

    Returns:
        A pandas DataFrame with columns 'title', 'summary', and 'date'.
    """

    paper = newspaper.build(url)
    articles = paper.articles

    data = []
    # n = 0
    for article in articles:
        # if n > 30:
        #     break
        try:   
            article.download()
            article.parse()
            text = article.text
            summary = summarize(text)
            # language = detect(text)
        except:
            print('error occured')

        # print(language)
        data.append({'title': article.title,
                    'text': text,
                    'url': article.url,
                    'summary': summary})
        # n += 1

    df = pd.DataFrame(data)
    return df

def summarize(text):
    if type(text) == type(None):
        return None
    # Prepare the parser and summarizer
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()

    # Set the number of sentences you want in the summary
    summary = summarizer(parser.document, 3)  # Summary with 3 sentences

    sentenceV = ""
    # Print the summary sentences
    for sentence in summary:
        sentenceV += str(sentence)
    
    return sentenceV

In [6]:
# Replace with the actual Yahoo Finance URL for Tesla news
url = "https://stockanalysis.com/stocks/aapl/"

df = scrape_tesla_news(url)

# Save the DataFrame to a CSV file
df.to_csv('tesla_news.csv', index=False, mode='a')

error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured
error occured


In [12]:
from newspaper import Article

In [ ]:
#Method number one of getting date. Usually doesn't work
for i, url in enumerate(df['url']):
    try:
        article = Article(url)
        article.download()
        article.parse()
        df.at[i, 'date'] = article.publish_date
    except Exception as e:
        df.at[i, 'text'] = None
        df.at[i, 'date'] = None
        print('error occured', e)

error occured Article `download()` failed with 403 Client Error: Forbidden for url: https://www.barrons.com/articles/qualcomm-stock-apple-amd-intel-chips-7c82508f on URL https://www.barrons.com/articles/qualcomm-stock-apple-amd-intel-chips-7c82508f
error occured Article `download()` failed with 403 Client Error: Forbidden for url: https://seekingalpha.com/article/4738832-my-dividend-stock-portfolio-new-october-dividend-record-100-holdings-with-15-buys?source=affiliate_program:stockanalysis.com&utm_medium=affiliate&utm_source=stockanalysis.com&affid=858&oid=16&transaction=5657c4b954ac41c69e9b0189a68cdc88 on URL https://stockanalysis.com/out/news?url=https://seekingalpha.com/article/4738832-my-dividend-stock-portfolio-new-october-dividend-record-100-holdings-with-15-buys
error occured Article `download()` failed with 401 Client Error: HTTP Forbidden for url: https://www.reuters.com/technology/apple-sends-100-mln-investment-proposal-build-plant-indonesia-2024-11-20/ on URL https://www.reu

In [17]:
from bs4 import BeautifulSoup as bs
import requests

In [18]:
#Getting the dates (r u sure this wroks...?)
#If the website is working with beautiful soup, can't we just get the urls with beautiful soup too? 
#beacause bs4 is way more lightweight than newspaper3k
r = requests.get('https://stockanalysis.com/stocks/aapl/')
soup = bs(r.text, 'html.parser')

dates = soup.select('div.mt-1.text-sm.text-faded.sm\\:order-1.sm\\:mt-0')

#Are we sure about the accuray of this?
all_dates = []

for date in dates:
    all_dates.append(date['title'])

In [19]:
#appending the date
df["date"] = pd.Series(all_dates)

In [22]:
df.tail()

,title,text,url,summary,date
64,Nvidia revenue forecast falls short of top est...,Nvidia (NVDA) reported third-quarter results t...,https://invezz.com/news/2024/11/20/nvidia-reve...,Nvidia (NVDA) reported third-quarter results t...,NaN
65,Nvidia shares take fresh hit as growth expecta...,"Nvidia, the chipmaker driving the boom in arti...",https://news.sky.com/story/nvidia-shares-take-...,Some would argue it is a nice problem to have ...,NaN
66,Nvidia earnings: AI chip leader shows no signs...,"The AI chipmaker Nvidia, the world’s most valu...",https://www.theguardian.com/technology/2024/no...,But many of Nvidia’s chip-making peers have be...,NaN
67,Boeing gives lay off notices to nearly 700 in ...,"The AI chipmaker Nvidia, the world’s most valu...",https://www.reuters.com/business/aerospace-def...,But many of Nvidia’s chip-making peers have be...,NaN
68,Reddit deploys fix for outage affecting thousa...,"The AI chipmaker Nvidia, the world’s most valu...",https://www.reuters.com/technology/reddit-depl...,But many of Nvidia’s chip-making peers have be...,NaN


In [39]:
type(df.iloc[-1,4]) == type(np.nan)

True

In [45]:
import numpy as np
#getting which websites are fucked
df[df['date'].apply(type) == type(np.nan)].shape

(44, 5)

In [4]:
df = pd.read_csv('tesla_news.csv')

In [15]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [13]:
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone", num_labels=3)

In [14]:
X = df['summary'].to_list()
labels = {0:'neutral', 1:'positive',2:'negative'}

In [16]:
sent_val = list()
for x in X:
    inputs = tokenizer(x, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs).logits
   
    probs = torch.nn.functional.softmax(outputs, dim=-1)
    val = labels[np.argmax(outputs.detach().numpy())]
    
    score_dict = {
        'neutral': probs[0][0].item(),
        'positive': probs[0][1].item(),
        'negative': probs[0][2].item()
    }
    
    sent_val.append(score_dict)

In [18]:
sent_val

[{'neutral': 0.04990115016698837,
  'positive': 0.9471642971038818,
  'negative': 0.0029345459770411253},
 {'neutral': 0.05384838581085205,
  'positive': 0.08265241235494614,
  'negative': 0.8634992241859436},
 {'neutral': 0.10224691778421402,
  'positive': 0.002626292407512665,
  'negative': 0.8951267600059509},
 {'neutral': 0.10224691778421402,
  'positive': 0.002626292407512665,
  'negative': 0.8951267600059509},
 {'neutral': 0.10224691778421402,
  'positive': 0.002626292407512665,
  'negative': 0.8951267600059509},
 {'neutral': 0.10224691778421402,
  'positive': 0.002626292407512665,
  'negative': 0.8951267600059509},
 {'neutral': 0.10224691778421402,
  'positive': 0.002626292407512665,
  'negative': 0.8951267600059509},
 {'neutral': 0.9322450757026672,
  'positive': 0.006859379354864359,
  'negative': 0.060895539820194244},
 {'neutral': 0.002832503989338875,
  'positive': 0.9967838525772095,
  'negative': 0.0003836597898043692},
 {'neutral': 0.9605203866958618,
  'positive': 0.003

In [19]:
neural_list = []
positive_list = []
negative_list = []

for item in sent_val:
    neural_list.append(item['neutral'])
    positive_list.append(item['positive'])
    negative_list.append(item['negative'])

In [22]:
df['neutral'] = neural_list
df['positive'] = positive_list
df['negative'] = negative_list

In [ ]:
df.head()

,title,text,url,summary,neutral,positive,negative
0,Latest IPO News,"Stocks took a breather on Thursday (Nov. 14), ...",https://stockanalysis.com/ipos/news/,"Stocks took a breather on Thursday (Nov. 14), ...",0.049901,0.947164,0.002935
1,Apple Seeks Dismissal of U.S. Smartphone Monop...,Apple wants the dismissal of a case accusing i...,https://www.pymnts.com/apple/2024/apple-seeks-...,The DOJ has accused Apple of preventing compet...,0.053848,0.082652,0.863499
2,Apple Urges Mac Users to Update After Hackers ...,Apple is recommending that Mac users update th...,https://www.cnet.com/tech/services-and-softwar...,"In the first instance, Apple said one issue st...",0.102247,0.002626,0.895127
3,Qualcomm Plans for Life After Apple. It's Comi...,Apple is recommending that Mac users update th...,https://www.barrons.com/articles/qualcomm-stoc...,"In the first instance, Apple said one issue st...",0.102247,0.002626,0.895127
4,My Dividend Stock Portfolio: New October Divid...,Apple is recommending that Mac users update th...,https://stockanalysis.com/out/news?url=https:/...,"In the first instance, Apple said one issue st...",0.102247,0.002626,0.895127


In [ ]:
# df.to_csv('tesla_news.csv', index=False)

In [25]:
df['positive'].mean()

np.float64(0.24119636888876467)

In [26]:
df['negative'].mean()

np.float64(0.22940303678726365)

In [27]:
df['neutral'].mean()

np.float64(0.5294005910050238)